# Simulation of 2D mail delivery system with Monte Carlo
> By **Team EpicCool** (Zayan, Kabir and Prannaya)<br>
> _8 December 2021_

IN PYTHON

In [35]:
import np
import pandas as pd
import matplotlib.pyplot as plt
from random import shuffle

In [2]:
def connect_matrix_2D():
    #1st row is source, last row is drain. all entries are connected to source, all exits are connected to drain!
    C = np.zeros((8,8));
    C[0,1] = 1;
    C[1,4] = 1;
    C[1,2] = 1;
    C[4,5] = 1;
    C[2,5] = 1;
    C[2,3] = 1;
    C[5,6] = 1;
    C[6,7] = 1;
    C[3,7] = 1;
    C[4,1] = 1;
    C[2,1] = 1;
    C[5,4] = 1;
    C[5,2] = 1;
    C[3,2] = 1;
    C[6,5] = 1;

    return C

In [3]:
def connect_matrix_A():
    A = np.zeros((8,8));
    A[0,1] = 1;
    
    A[1,4] = 0.9;
    A[1,2] = 0.1;
    
    A[4,5] = 0.9;
    A[2,5] = 0.1;
    
    A[5,6] = 0.7;
    A[5,4] = 0.2;
    A[5,2] = 0.1;
    
    A[2,5] = 0.9;
    A[2,3] = 0;
    A[2,1] = 0.1;
    
    A[3,7] = 0;
    A[3,2] = 1;
    
    A[6,7] = 1;
    A[6,5] = 0;
    return A


In [4]:
def connect_matrix_B():
    B = np.zeros((8,8));
    B[0,1] = 1;
    
    B[1,4] = 0.1;
    B[1,2] = 0.9;
    
    B[4,5] = 0.1;
    B[4,1] = 0.9;
    
    B[5,6] = 0;
    B[5,4] = 0.1;
    B[5,2] = 0.9;
    
    B[2,5] = 0.1;
    B[2,3] = 0.7;
    B[2,1] = 0.2;
    
    B[3,7] = 1;
    B[3,2] = 0;
    
    B[6,7] = 0;
    B[6,5] = 1;
    return B


In [11]:
def initialize_H(NC,tM):
    # tM maximum number of time steps
    # NC is the number of mails
    return np.zeros((NC, tM+2), dtype=np.uint8); #this tM+2 can be alterd to taste    

In [97]:
def find_pos_available_k(ic,it,H,C,L,conn_mat):
    ip = H[ic,it]
    reachable_positions = C[ip,:]
    vector_probabilities = conn_mat[ip,:]
    print(reachable_positions, vector_probabilities)
    occupied_positions = H[:,it]
    print(occupied_positions)
    reachable_positions[occupied_positions] = 0
    vector_positions = np.argwhere(reachable_positions==1).T[0]
    vector_probabilities = vector_probabilities * reachable_positions
    print(reachable_positions, vector_probabilities)
    if vector_probabilities.sum():
        vector_probabilities = (vector_probabilities / vector_probabilities.sum())[vector_positions]
    else: vector_probabilities, vector_positions = np.array([]), np.array([])
    print(vector_positions, vector_probabilities)
    return vector_positions, vector_probabilities


In [84]:
H = initialize_H(4, 10)
H[0,0] = 7;
H[1,0] = 5;
H[2,0] = 3;
C = connect_matrix_2D()
conn_matA = connect_matrix_A()
vec_pos, Avec_prob = find_pos_available_k(1,0,H,C,6,conn_matA)
vec_pos, Avec_prob

(array([2, 4, 6], dtype=int64), array([0.1, 0.2, 0.7]))

In [53]:
H = initialize_H(4, 10)
H[0,0] = 7;
H[1,0] = 5;
H[2,0] = 3;
C = connect_matrix_2D()
conn_matA = connect_matrix_A()
vec_pos, Avec_prob = find_pos_available_k(1,0,H,C,6,conn_matA)
vec_pos, Avec_prob

(array([2, 4, 6], dtype=int64), array([0.1, 0.2, 0.7]))

In [76]:
def move_position_k(ic,it,H,vec_pos, Avec_prob):
    result = np.random.choice(vec_pos, 1, p=Avec_prob)[0] if len(vec_pos) else H[ic,it]
    H[ic,it+1]=result
    return H

In [63]:
def deliver_mail_k(ic,numA,it,H,C,L,conn_matA,conn_matB):
    H = move_position_k(ic,it,H,*find_pos_available_k(ic,it,H,C,L,conn_matA if ic < numA else conn_matB))

In [73]:
def find_resolve_conflicts(H,L,it):
    for iL in range(1,L+1):
        conflicts = np.argwhere(H[:,it]==iL).T[0]
        if len(conflicts) > 1:
            np.random.shuffle(conflicts)
            choices_go_back = conflicts[:-1]
            H[choices_go_back, it] = H[choices_go_back, it-1]

    return H

In [88]:
def next_step_k(H,it,C,L,numA,conn_matA,conn_matB):
    print(H)
    for ic in range(len(H)):
        H = deliver_mail_k(ic,numA,it,H,C,L,conn_matA,conn_matB)
        print(H)
    H = find_resolve_conflicts(H, L, it+1)
    print(H)
    return H


In [98]:
NMail = 4; # number of mails
tM = 3; # total time steps
L = 6; # number of sites
numA = 2 # mail A number
numB = NMail - numA # mail B number

C = connect_matrix_2D();
conn_matA = connect_matrix_A()
conn_matB = connect_matrix_B()
H = initialize_H(NMail,tM);

for it in range(1,tM+1):
    H = next_step_k(H,it,C,L,numA,conn_matA,conn_matB);

H[0]

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[0. 1. 0. 0. 0. 0. 0. 0.] [0. 1. 0. 0. 0. 0. 0. 0.]
[0 0 0 0]
[0. 1. 0. 0. 0. 0. 0. 0.] [0. 1. 0. 0. 0. 0. 0. 0.]
[1] [1.]
[[0 0 1 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[0. 1. 0. 0. 0. 0. 0. 0.] [0. 1. 0. 0. 0. 0. 0. 0.]
[0 0 0 0]
[0. 1. 0. 0. 0. 0. 0. 0.] [0. 1. 0. 0. 0. 0. 0. 0.]
[1] [1.]
[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[0. 1. 0. 0. 0. 0. 0. 0.] [0. 1. 0. 0. 0. 0. 0. 0.]
[0 0 0 0]
[0. 1. 0. 0. 0. 0. 0. 0.] [0. 1. 0. 0. 0. 0. 0. 0.]
[1] [1.]
[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 0 0]]
[0. 1. 0. 0. 0. 0. 0. 0.] [0. 1. 0. 0. 0. 0. 0. 0.]
[0 0 0 0]
[0. 1. 0. 0. 0. 0. 0. 0.] [0. 1. 0. 0. 0. 0. 0. 0.]
[1] [1.]
[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]]
[[0 0 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[[0 0 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[0. 1. 0. 0. 0. 0. 0. 0.] [0. 1. 0. 0. 0. 0. 0. 0.]
[0 1 0 0]
[0. 0. 0. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 0. 0.]
[] []
[[0 0 0 0 0]
 [0 

array([0, 0, 0, 0, 0], dtype=uint8)